In [1]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

import matplotlib.pylab as plt
%matplotlib inline

In [2]:
# Training settings
batch_size = 64
test_batch_size = 128
epochs = 10
lr = 0.01
momentum = 0.5
no_cuda = True
seed = 1
log_interval = 100

cuda = not no_cuda and torch.cuda.is_available()

In [3]:
torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

In [11]:
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.softmax(x)

In [13]:
model = Net()
if cuda:
    model.cuda()

In [14]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [15]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))

In [16]:
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if cuda:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [19]:
for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 1.638289
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.700839
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.745248
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.682550
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.558452
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1.717279
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1.663266
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1.634116
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.654273
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1.649224

Test set: Average loss: -0.8611, Accuracy: 8628/10000 (86%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 1.542576
Train Epoch: 2 [6400/60000 (11%)]	Loss: 1.654288
Train Epoch: 2 [12800/60000 (21%)]	Loss: 1.651888
Train Epoch: 2 [19200/60000 (32%)]	Loss: 1.622441
Train Epoch: 2 [25600/60000 (43%)]	Loss: 1.552174
Train Epoch: 2 [32000/60000 (53%)]	Loss: 1.592058
Train Epoch: 2 [38400/60000 (64%)]	Loss: 1.598593
Train Epoch: 2 [44800/60000 (75%)]	Loss: 1.594854
Train Epoch: 2 [51200/60000 (85%)]	Loss: 1.653771


In [20]:
torch.save(model, 'mnist_model.pt')
#model = torch.load('mnist_model.pt')

/home/egor/anaconda3/lib/python3.5/site-packages/torch/serialization.py:147: UserWarning: Couldn't retrieve source code for container of type Net. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [15]:
torch.load('mnist_model.pt')

Net (
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d (p=0.5)
  (fc1): Linear (320 -> 50)
  (fc2): Linear (50 -> 10)
)